In [1]:
import pandas as pd
import numpy as np
data_path = "/Users/rose/Documents/GitHub/The 4th-ML100Days/data/Day7/"
df_train = pd.read_csv(data_path+"house_train.csv")
df_test = pd.read_csv(data_path+"house_test.csv")

In [2]:
df_train.shape

(1460, 81)

In [3]:
train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [4]:
dtype_df = df.dtypes.reset_index() 
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,25
1,float64,11
2,object,43


In [5]:
# 確定只有 int64, float64, object 三種類型後對欄位名稱執行迴圈, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
# .dtypes(欄位類型), .columns(欄位名稱) 是 DataFrame 提供的兩個方法, 這裡順便展示一下 for 與 zip 搭配的用法
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
# 這邊採用的寫法稱為 f-string, 是 Python 3.6.2 以後版本才出現的
# 如果無法執行, 則需要更新到這個版本之後, 或自行將程式改寫為 str.format 形式
# 改寫方式可以參考 https://blog.louie.lu/2017/08/08/outdate-python-string-format-and-fstring/
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

25 Integer Features : ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

11 Float Features : ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

43 Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fi

In [6]:
df[float_features].head()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0
1,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0
2,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0
3,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0
4,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0


In [7]:
# 作業一
float_mean= df[float_features].mean()
float_max= pd.concat([float_mean, df[float_features].max()], axis = 1)
float_mix= pd.concat([float_max, df[float_features].nunique()], axis = 1)
float_mix.columns = ["mean", "max", "nunique"]
float_mix

,mean,max,nunique
LotFrontage,69.305795,313.0,128
MasVnrArea,102.201312,1600.0,444
BsmtFinSF1,441.423235,5644.0,991
BsmtFinSF2,49.582248,1526.0,272
BsmtUnfSF,560.772104,2336.0,1135
TotalBsmtSF,1051.777587,6110.0,1058
BsmtFullBath,0.429894,3.0,4
BsmtHalfBath,0.061364,2.0,3
GarageYrBlt,1978.113406,2207.0,103
GarageCars,1.766621,5.0,6


In [8]:
int_mean= df[int_features].mean()
int_max= pd.concat([int_mean, df[int_features].max()], axis = 1)
int_mix= pd.concat([int_max, df[int_features].nunique()], axis = 1)
int_mix.columns = ["mean", "max", "nunique"]
int_mix

,mean,max,nunique
MSSubClass,57.137718,190,16
LotArea,10168.114080,215245,1951
OverallQual,6.089072,10,10
OverallCond,5.564577,9,9
YearBuilt,1971.312778,2010,118
YearRemodAdd,1984.264474,2010,61
1stFlrSF,1159.581706,5095,1083
2ndFlrSF,336.483727,2065,635
LowQualFinSF,4.694416,1064,36
GrLivArea,1500.759849,5642,1292


In [9]:
object_mean= df[object_features].mean()
object_max= pd.concat([object_mean, df[object_features].max()], axis = 1)
object_mix= pd.concat([object_max, df[object_features].nunique()], axis = 1)
object_mix

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,0
Alley,NaN,NaN,2
BldgType,NaN,NaN,5
BsmtCond,NaN,NaN,4
BsmtExposure,NaN,NaN,4
BsmtFinType1,NaN,NaN,6
BsmtFinType2,NaN,NaN,6
BsmtQual,NaN,NaN,4
CentralAir,NaN,NaN,2
Condition1,NaN,NaN,9
Condition2,NaN,NaN,8


In [10]:
#在float及int資料型態都能夠使用數值運算mean及max函數，但在object則會因為是文字型類別變數無法計算
#而nunique函數是只有計算變數個數所以三種型態皆可使用

In [11]:
#作業二
#圖片型態資料，會歸類在數值型變數做處理
#最複雜為類別型變數，因需考慮較多狀況做處理，比如需使用dummy variable或是one-hot-coding，以及遇到排序型的特徵，有時會以類別型變數呈現進而模糊處理方式